In [ ]:
from google.colab import drive
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import re #Regular Expression
import ast
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
gold_corpus = open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/gold.txt', encoding='utf-8').readlines()
len(gold_corpus)

2211

In [ ]:
manual_tokens = open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/manual_tokens.txt', encoding='utf-8').readlines()

In [ ]:
new_line_idx = [i for i, item in enumerate(manual_tokens) if item == '\n']

with open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/train_gold.txt', 'w', encoding='utf-8') as f:
    for line in gold_corpus[:new_line_idx[59]]: 
      f.write(line)
    f.write('\n')
    
with open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/test_gold.txt', 'w', encoding='utf-8') as f:
    for line in gold_corpus[new_line_idx[59] + 1:]: 
      f.write(line)

with open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/train_words.txt', 'w', encoding='utf-8') as f:
    for line in gold_corpus[:new_line_idx[59]]: 
      f.write(re.sub('\t.*', '', line))
    f.write('\n')        
    
with open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/test_words.txt', 'w', encoding='utf-8') as f:
    for line in gold_corpus[new_line_idx[59] + 1:]: 
      f.write(re.sub('\t.*', '', line))

In [ ]:
def preprocess(vocabs_dict, path):
    data = []
    file = open(path, encoding='utf-8').readlines()
    
    for index, word in enumerate(file):
        if not word.split():
            word = '--n--'
            data.append(word)
            continue
        elif word.strip() not in vocabs_dict:
            word = '--unk--'
            data.append(word)
            continue
        data.append(word.strip())
    return data

def plot_tag_counts(gold):
    tags = [word_tag.split()[1] for word_tag in gold if word_tag.split()]
    tag_counts = pd.DataFrame(tags)[0].value_counts()
    tag_counts.plot.bar(rot=0, width=0.7, legend=False, figsize=(15, 5))
    return pd.DataFrame(tag_counts).T.assign(Total=tag_counts.sum())

In [ ]:
vocabs = open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/vocabs.txt', encoding='utf-8').read().split('\n')
vocabs_dict = {}
index = 0

for word in sorted(vocabs): 
    if word not in vocabs_dict: 
        vocabs_dict[word] = index  
        index += 1

In [ ]:
train_gold = open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/train_gold.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập train_gold:', len(train_gold))

train_words = preprocess(vocabs_dict, '/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/train_words.txt')
print('Số lượng từ trong tập train_words:', len(train_words))

test_gold = open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/test_gold.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập test_gold:', len(test_gold))

test_words = preprocess(vocabs_dict, '/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/test_words.txt')
print('Số lượng từ trong tập test_words:', len(test_words))

Số lượng từ trong tập train_gold: 1709
Số lượng từ trong tập train_words: 1709
Số lượng từ trong tập test_gold: 502
Số lượng từ trong tập test_words: 502


In [ ]:
!pip install vncorenlp
!pip install py_vncorenlp
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=e981730aa40ce34c2d87b74996eeeac01f648981eda8daf01bc6caa55adbc71a
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.7 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=62e6ca655cf45386df47d2964cec086f5bde49df58cf5babf2b412fbc779eb35
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827


In [ ]:
import py_vncorenlp
from pyvi import ViPosTagger
py_vncorenlp.download_model()
client = py_vncorenlp.VnCoreNLP(annotators=["pos"])

In [ ]:
test_words = open('/content/gdrive/MyDrive/Colab Notebooks/CS 212 - NLP/Đồ án/test_words.txt', 'r', encoding='utf-8').readlines()

In [ ]:
from sklearn.metrics import classification_report
#client = py_vncorenlp.VnCoreNLP(annotators=["pos"])
def report(client):
    y_pred = []
    y_true = []
    for word in test_words:
        word = word.replace('\n', '')
        if '_' not in word:
          predict = client.annotate_text(word)
        else: 
          predict = client.annotate_text(word.replace('_', ' '))
        if predict != {}:
          y_pred.append(predict[0][0]["posTag"])
        else:
          continue

    for word in test_gold:
        word_tag_tuple = word.split()
        if len(word_tag_tuple) != 2:
          continue 
        word, tag = word_tag_tuple
        y_true.append(tag)
    print(classification_report(y_true, y_pred, zero_division=0))
    return y_pred, y_true

print('Kết quả khi sử dụng thư viện VnCoreNLP trên tập test:\n')
y_pred, y_true_test = report(client)

Kết quả khi sử dụng thư viện VnCoreNLP trên tập test:

              precision    recall  f1-score   support

           A       1.00      1.00      1.00        33
           C       1.00      0.64      0.78        11
          CH       1.00      1.00      1.00        45
          Cc       1.00      1.00      1.00         9
           E       0.89      1.00      0.94        32
           L       1.00      1.00      1.00        12
           M       1.00      1.00      1.00        13
           N       0.98      0.95      0.97       133
          Np       0.74      1.00      0.85        25
          Nu       0.00      0.00      0.00         1
          Ny       1.00      0.43      0.60         7
           P       0.96      1.00      0.98        22
           R       0.96      0.96      0.96        24
           T       1.00      1.00      1.00         2
           V       0.99      0.99      0.99        96
           X       1.00      0.83      0.91         6
           Z       1.00   

In [ ]:
client_2 = ViPosTagger
def report(client):
    y_pred = []
    y_true = []
    for word in test_words:
        word = word.replace('\n', '')
        predict = client_2.postagging(word)
        if predict[1][0] == 'F':
          predict[1][0] = 'CH'
        if predict[1][0] == 'C':
          predict[1][0] = 'Cc'
        if predict[1][0] == 'S':
          predict[1][0] = 'C'
        if word != "":
          y_pred.append(predict[1][0])
        else:
          continue

    for word in test_gold:
        word_tag_tuple = word.split()
        if len(word_tag_tuple) != 2:
          continue 
        word, tag = word_tag_tuple
        y_true.append(tag)
    print(classification_report(y_true, y_pred, zero_division=0))
    return y_pred, y_true

print('Kết quả khi sử dụng thư viện Pyvi trên tập test:\n')
y_pred, y_true_test = report(client_2)

Kết quả khi sử dụng thư viện Pyvi trên tập test:

              precision    recall  f1-score   support

           A       0.72      0.94      0.82        33
           C       0.00      0.00      0.00        11
          CH       1.00      1.00      1.00        45
          Cc       0.64      1.00      0.78         9
           E       1.00      0.03      0.06        32
           L       1.00      1.00      1.00        12
           M       1.00      0.77      0.87        13
           N       0.73      0.97      0.83       133
          Np       0.57      0.80      0.67        25
          Nu       0.00      0.00      0.00         1
          Ny       1.00      0.29      0.44         7
           P       1.00      0.09      0.17        22
           R       0.71      1.00      0.83        24
           T       0.00      0.00      0.00         2
           V       0.87      0.88      0.87        96
           X       0.00      0.00      0.00         6
           Z       0.00      0.